<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
# !gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
# !unzip archive.zip

Импортируем нужные библиотеки:

In [2]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [3]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [5]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
data.emotion.nunique(), data.flag.nunique(), data.user.nunique()

(2, 1, 659775)

Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [7]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [8]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [9]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print("\n".join(line))

@
claire_nelson
i
'
m
on
the
north
devon
coast
the
next
few
weeks
will
be
down
in
devon
again
in
may
sometime
i
hope
though
!


In [10]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [11]:
import os 
from gensim.models import KeyedVectors
from gensim.downloader import base_dir



In [12]:
path = os.path.join(base_dir, 'glove-twitter-100', 'glove-twitter-100.gz')
word2vec = KeyedVectors.load_word2vec_format(path) # api.load("fasttext-wiki-news-subwords-300")

In [13]:
word2vec

In [14]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(100,)


In [15]:
len(emb_line[0])

100

Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [16]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(100,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [367]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    
    
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

        
    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    
    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokenizer = nltk.WordPunctTokenizer()
        tokens = tokenizer.tokenize(text.lower())
        filtered_tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        
        return filtered_tokens

        
    def get_embeddings_(self, tokens):
        embeddings = [word2vec.get_vector(w) for w in tokens if w in word2vec] # Получи эмбеддинги слов и усредни их

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [368]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [369]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [370]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


# pca = PCA(n_components=2)
# examples["transformed_features"] = pca.fit_transform(examples['features'])# Обучи PCA на эмбеддингах слов

tsne = TSNE(n_components=2)
examples["transformed_features"] = tsne.fit_transform(examples['features'])

In [371]:
# !pip install bokeh

In [372]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [373]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    );

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [374]:
len(dev)

1280000

In [375]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [376]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    mean_acc = 0
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получи предсказания модели
        y_pred = model(features)
        y_pred = y_pred.reshape(-1)
        # Посчитай лосс
        loss = criterion(y_pred, targets.float()) 
        # Обнови параметры модели
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()

        acc = accuracy_score(targets.cpu().numpy(), y_pred.detach().cpu().numpy() > 0.5)
        mean_acc += acc.item()
        
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
                             
    pbar.set_description(f"Train Acc: {mean_acc / len(train_loader):.4}")

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            y_pred = model(features)
            y_pred = y_pred.reshape(-1)
            # Посчитай лосс
            loss = criterion(y_pred, targets.float())  
                             
            acc = accuracy_score(targets.cpu().numpy(), y_pred.cpu().numpy() > 0.5) # Посчитай точность модели
            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

In [377]:
word2vec.vector_size

100

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [378]:
dev.word2vec.vector_size

100

In [379]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-3
num_epochs = 5

model = nn.Sequential(
    nn.Linear(vector_size, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, num_classes),
    nn.Sigmoid())
model = model.cuda()
criterion = nn.BCELoss()# Твой лосс
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [380]:
#testing(model, criterion, valid_loader, device)

In [381]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5125732548236847, 'Test Acc': 0.744921875}



{'Test Loss': 0.5070803210735321, 'Test Acc': 0.74793359375}



{'Test Loss': 0.5048438949584961, 'Test Acc': 0.74902734375}



{'Test Loss': 0.5050263856649398, 'Test Acc': 0.7492734375}



{'Test Loss': 0.5058892072439194, 'Test Acc': 0.7483671875}


In [382]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5072701189655084, 'Test Acc': 0.7472356729233227}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

# Sum of context embeddings

In [414]:
class TwitterDataset_1(Dataset):
    
    
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

        
    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    
    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokenizer = nltk.WordPunctTokenizer()
        tokens = tokenizer.tokenize(text.lower())
        filtered_tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        
        return filtered_tokens

        
    def get_embeddings_(self, tokens):
        w_size = 3
        embeddings = []

        for i in range(len(tokens)):
            if tokens[i] in self.word2vec:
                embeddings.append(self.word2vec.get_vector(tokens[i]))
            else:
                emb_sum = np.zeros((2 * w_size, self.word2vec.vector_size))
                left_idx = i - w_size
                right_idx = i + w_size
                
                # Check the left side of the window inside the line, otherwise pad it
                if left_idx < 0:
                    left_padding = np.zeros((abs(left_idx), self.word2vec.vector_size))
                    left_emb = np.zeros((i, self.word2vec.vector_size))
                    for idx in range(0, i):
                        if tokens[idx] in word2vec:
                            left_emb[idx] = self.word2vec.get_vector(tokens[idx])
                    emb_sum[:w_size] += np.vstack((left_padding, left_emb))
                    
                # Check the right side also
                if right_idx > len(tokens) - 1:
                    right_padding = np.zeros((abs(right_idx) - len(tokens) + 1, self.word2vec.vector_size))
                    right_emb = np.zeros((len(tokens) - (i + 1), self.word2vec.vector_size))
                    for idx in range(i + 1, len(tokens)):
                        if tokens[idx] in word2vec:
                            right_emb[idx - (i + 1)] = self.word2vec.get_vector(tokens[idx])
                    emb_sum[w_size:] += np.vstack((right_emb, right_padding))
                    
                emb_sum /= 2 * w_size 
                weights = np.array([1 / abs(w_size - j) for j in range(w_size)])
                weights = np.hstack((weights, weights[::-1]))

                # Let's sum neighbors embeddings with weights=1/N
                emb_sum = weights @ emb_sum
                
                embeddings.append(emb_sum)
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
                
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [415]:
dev = TwitterDataset_1(dev_data, "text", "emotion", word2vec)

In [416]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::500]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

2560


In [417]:
tsne = TSNE(n_components=2)
examples["transformed_features"] = tsne.fit_transform(examples['features'])

In [418]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [419]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    );

In [420]:
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [421]:
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-3
num_epochs = 5

model = nn.Sequential(
    nn.Linear(vector_size, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, num_classes),
    nn.Sigmoid())
model = model.cuda()
criterion = nn.BCELoss()# Твой лосс
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

In [422]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5138566032648086, 'Test Acc': 0.74311328125}



{'Test Loss': 0.5090950635671616, 'Test Acc': 0.74621875}



{'Test Loss': 0.5047427283525467, 'Test Acc': 0.7496015625}



{'Test Loss': 0.5076169843673706, 'Test Acc': 0.74712109375}



{'Test Loss': 0.5008482927083969, 'Test Acc': 0.75103125}


In [423]:
test_loader = DataLoader(
    TwitterDataset_1(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.501602554283203, 'Test Acc': 0.7512355231629393}


# Tf-idf

In [265]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [341]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dev_data['text'])

print(X.shape)

(1280000, 589604)


In [342]:
#!pip install sparsesvd

In [343]:
from sparsesvd import sparsesvd
import scipy
from scipy.spatial import distance

In [344]:
ut, s, vt = sparsesvd(scipy.sparse.csc_matrix(X), 100) # do SVD, asking for 100 factors

In [395]:
vt.shape

(100, 589604)

In [396]:
tf_idf_emb = {}
i = 0
for w in vectorizer.get_feature_names():
    tf_idf_emb[w] = vt[:, i]
    i += 1

In [397]:
'cat' in tf_idf_emb

True

### Посмотрим, что tf-idf получился разумным

In [398]:
1 - distance.cosine(tf_idf_emb['dog'], tf_idf_emb['dog'])

1.0

In [399]:
1 - distance.cosine(tf_idf_emb['dog'], tf_idf_emb['cat'])

0.9225991767080144

In [400]:
1 - distance.cosine(tf_idf_emb['dog'], tf_idf_emb['apple'])

0.15761865915595763

In [401]:
class TwitterDataset_2(Dataset):
    
    
    def __init__(self, data: 
                 pd.DataFrame, 
                 feature_column: str, 
                 target_column: str, 
                 word2vec: gensim.models.Word2Vec, 
                 tf_idf: dict
                ):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tf_idf = tf_idf

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

        
    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    
    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokenizer = nltk.WordPunctTokenizer()
        tokens = tokenizer.tokenize(text.lower())
        filtered_tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        
        return filtered_tokens

        
    def get_embeddings_(self, tokens):
        w_size = 3
        embeddings = []

        for i in range(len(tokens)):
            if tokens[i] in self.word2vec and tokens[i] in self.tf_idf:
                embeddings.append(self.word2vec.get_vector(tokens[i]) + self.tf_idf[tokens[i]])
            elif tokens[i] in self.tf_idf:
                embeddings.append(self.tf_idf[tokens[i]])
        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
                
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [358]:
dev = TwitterDataset_2(dev_data, "text", "emotion", word2vec, tf_idf_emb)

In [402]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [403]:
tsne = TSNE(n_components=2)
examples["transformed_features"] = tsne.fit_transform(examples['features'])

In [404]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [405]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    );

In [406]:
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [407]:
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-3
num_epochs = 5

model = nn.Sequential(
    nn.Linear(vector_size, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, num_classes),
    nn.Sigmoid())
model = model.cuda()
criterion = nn.BCELoss()# Твой лосс
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

In [412]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.49988570749759675, 'Test Acc': 0.751984375}



{'Test Loss': 0.4996106719970703, 'Test Acc': 0.752546875}



{'Test Loss': 0.4982139586210251, 'Test Acc': 0.75325390625}



{'Test Loss': 0.4979131450653076, 'Test Acc': 0.75329296875}



{'Test Loss': 0.49811078083515165, 'Test Acc': 0.7535625}


In [413]:
test_loader = DataLoader(
    TwitterDataset_2(test_data, "text", "emotion", word2vec, tf_idf_emb), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5036755599343358, 'Test Acc': 0.7505148013178914}


# Выводы
## Что я делал
### 1. Word2vec
Тут все понятно, делал так, как было написано авторами. Если посмотреть на графики семплов, то там довольно все перемешано и на глаз вообще с трудом можно что-то отделить. Но как бы и подход максимально простой. Тем ни менее, скор заметно выше, чем случайное угадывание. 

Так же пробовал играться с параметрами сети: число слоев, функции активации, различные оптимизаторы. Слишком сложная сетка с большим количеством слоев не дает никакого выигрыша. Разве что значительное увеличения времени обучения. Лучше всего Adam и ReLU. По классике. 

*Замечаение.* В дальнейшем сетка будет таже самая на всех подходов, чтобы сравнение подходов к представлениям слов было корректным.

Результаты на тестовых данных: 'Test Acc': `0.7472`

### 2. Эмбеддинги соседей

Тут я пробовал различные длины окон: 1, 2, 3, 5, 10. Почти никакой разницы между ними я заметить не смог. Если брать слишком большой или слишком маленький размер окна, то это ничего не дает. Поэтому я взял размер окна 3. Веса для соседних эмбеддингов слов я брал обратно пропорционально их порядковому номеру от центрального слова. Например, если окно длины 3, то третье слово справа (и слева) будет делиться на 3 (точнее его эмбеддинг). И т.д. 

*Замечаение.* 

Результаты на тестовых данных: 'Test Acc': `0.7512`

### 3. Tf- Idf

Тут я, как было сказано, воспользовался sklearn. Обучил TfidfVectorizer(min_df=3). Затем разложил его с помощью SVD разложения. Взял оттуда матрицу `vt`, которая для представляния слов в документе. Затем просто плюсовал два numpy массива (один word2vec, другой tf-idf) и получал таким образом вектор представления для одного слова. После чего, весь этот винегрет подавал сетке.

Результаты на тестовых данных: 'Test Acc': `0.7505`

## Подытожим

Второй подход, с эмбеддингами соседей, дал больший скор, относительно остальных моделей. По-видимому, данный результат обусловлен тем, что здесь мы все-таки давали сетке большей данных, т.к. больше не зануляли пропуски. Это очевидный плюс, по сравнению с первой моделью. Почему вторая модель лучше третьей, точно сказать сложно. Скорее всего, во tf-idf подходе мы перешли в немного другое пространство, прибавив к word2vec еще tf-idf. Мне вообще кажется, что данных подход не должен особо чего дать. Потому что плюсуя к каждому вектору еще один, мы ничего принципиально нового сетке не даем. Но это все еще лучше, чем бейзлан, потому что мы учитываем пропуски.

## Что можно сделать лучше
1. Взять размеры эмбеддингов не 100, а 300. У меня гугл коллаб вылетал постоянно по памяти, если я пытался учиться на 300, поэтому я решил остановиться на 100. 
2. Попробовать не складывать word2vec и tf-idf, а подавать их в сетку по отдельности (например тапл). Таким образом, у сетки будет больше фичей и она сможет выучить новые зависимости. Но минусы такого подхода, что мы сильно раздуваем количество фичей и нужно больше памяти (привет, коллаб).